<a href="https://colab.research.google.com/github/xcamden/youtube-to-birdsong/blob/main/youtube_to_birdsong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Dependencies


In [ ]:
!git clone https://github.com/xcamden/youtube-to-birdsong

!apt install sox

!sudo -H pip install --upgrade youtube-dl

!chmod +x /content/youtube-to-birdsong/csdr-install.sh

!/content/youtube-to-birdsong/csdr-install.sh

#update already exiting ffmpeg
!sudo add-apt-repository -y ppa:mc3man/trusty-media; sudo apt-get update; sudo apt-get install --only-upgrade ffmpeg

Cloning into 'youtube-to-birdsong'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 30 (delta 9), reused 27 (delta 9), pack-reused 0
Unpacking objects: 100% (30/30), done.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3
Suggested packages:
  file libsox-fmt-all
The following NEW packages will be installed:
  libmagic-mgc libmagic1 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-base libsox3 sox
0 upgraded, 8 newly installed, 0 to remove and 34 not upgraded.
Need to get 760 kB of archives.
After this operation, 6,717 kB of additional

# Content Generation


In [ ]:
#delete files from previous executions 
!rm -rf *.wav *.mp3 *.mp4 *.prof *.mov

#@markdown Insert the link to the YouTube video you want to download & make into birdsong. Try to find a slow paced video and avoid avoid music videos.

# Björk talking about her TV: https://www.youtube.com/watch?v=75WFTHpOw8Y
YouTube_Link= "https://www.youtube.com/watch?v=75WFTHpOw8Y" #@param {type:"string"}   

#@markdown Set resolution of downloaded video. YouTube downloads take a while, so keep the max resolution low if possible.
MaxVideoHeight = "720" #@param {type:"string"}  

#@markdown Reduce background noise in the video. Don't change this if you don't know what you're doing. Try not to go below 0.2 or above 0.3
Static_Reduce = "0.28" #@param {type:"string"}    

#download video
!youtube-dl -f "bestvideo[height<={MaxVideoHeight}]+bestaudio/best[height<={MaxVideoHeight}]" --recode-video mp4 -o vid {YouTube_Link}

#save audio from video
!ffmpeg -i vid.mp4 -vn cc-raw.wav

#Generate a sample of noise from the audio of the file:
#-t duration: the duration (h:m:s.ms) of audio segment to cut that represents the noise.
!ffmpeg -i cc-raw.wav -acodec pcm_s16le -ar 128k -vn -ss 00:00:00.0 -t 00:00:01.0 noiseaud.wav

#Generate a noise profile in sox:
!sox noiseaud.wav -n noiseprof noise.prof

#Clean the noise samples from the audio stream:
!sox cc-raw.wav cc.wav noisered noise.prof {Static_Reduce}

#convert audio from video into birdsong
#requires soX and csdr
!perl /content/youtube-to-birdsong/blackbird.pl -y

#add birdsounds to original video
!ffmpeg -i vid.mp4 -i birdson3.wav -map 0:v -map 1:a -c:v copy -shortest vid+newaudio.mp4 

# View new generated video with blackbird audio

In [ ]:
from IPython.display import HTML
from base64 import b64encode

print ("Download this at /content/vid+newaudio.mp4" )

mp4 = open('/content/vid+newaudio.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)



Download this at /content/vid+newaudio.mp4


In [ ]:
import librosa
import IPython.display as ipd

print ("Download this at /content/cc.mp4" )

print ("New Audio:")

x, sr = librosa.load('cc.wav') 
ipd.Audio(x, rate=sr)

# View downloaded video with original audio




In [ ]:
from IPython.display import HTML
from base64 import b64encode

print ("Download this at /content/vid+newaudio.mp4" )

mp4 = open('/content/vid.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
import librosa
import IPython.display as ipd

print ("Download this at /content/cc-raw.mp4" )

print ("Original video audio:")

x, sr = librosa.load('cc-raw.wav') 
ipd.Audio(x, rate=sr)